<a href="https://colab.research.google.com/github/JoseGarza1996/NeuralNetwork/blob/main/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0

SyntaxError: ignored

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from scipy import stats
import keras

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import cv2
import os

import tensorflow as tf
from google.colab import drive
#drive.mount('/content/drive')


from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.layers.experimental.preprocessing import RandomFlip
from keras.layers.experimental.preprocessing import RandomRotation
from keras.layers.experimental.preprocessing import RandomContrast
from keras.layers.experimental.preprocessing import RandomZoom
from keras.layers.experimental.preprocessing import Rescaling

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model


In [ ]:
!unzip /content/cap-4611-spring-21-assignment-5.zip

In [ ]:
train = '/content/images/images/train'
test = '/content/images/images/test'
train_meta = pd.read_csv('/content/assignment5_training_data_metadata.csv')
test_meta = pd.read_csv('/content/assignment5_test_data_metadata.csv')

I chose to drop the Stress-Smoking label since they seemed to have been outliers. Detecting stress smoking is needless.

In [ ]:
indexnames = (train_meta[train_meta['type'] == 'Stress-Smoking']).index
train_meta.drop(indexnames, inplace=True)

##I checked for missing values in the column types and repalced those variables with normal if that corresponding label was normal.

In [ ]:
keys = test_meta.loc[:, 'id']

In [ ]:
train_meta.loc[(train_meta['label'] == 'Normal'), 'type'] = 1
train_meta.loc[(train_meta['type'] == 'Virus'), 'type'] = 2
train_meta.loc[(train_meta['type'] == 'bacteria'), 'type'] = 3


In [ ]:
train_meta = train_meta.dropna(axis=0, subset = ['type'])

In [ ]:
train_images = []
for x in train_meta.index:
  filepath = train + '/' + train_meta.loc[x, 'image_name']
  image = cv2.resize(cv2.imread(filepath, 0), (256,256))
  train_images.append([train_meta.loc[x, 'id'],np.array(image), train_meta.loc[x, 'image_name'],train_meta.loc[x, 'type']])

In [ ]:
test_images = []
for x in test_meta.index:
  filepath = test + '/' + test_meta.loc[x, 'image_name']
  image = cv2.resize(cv2.imread(filepath, 0), (256,256))
  test_images.append([test_meta.loc[x, 'id'], np.array(image), test_meta.loc[x, 'image_name']])

In [ ]:
train_data = np.array([i[1] for i in train_images]).reshape(-1,256,256,1)
train_label = np.array([i[3] for i in train_images])
test_data = np.array([i[1] for i in test_images]).reshape(-1,256,256,1)

I attempted to design a model that would be sensitive to high level features. Dropout layers are to reduce overfitting sincce the data is so limited.

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape = (256,256,1)))

model.add(Conv2D(filters=256, kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=256, kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=512, kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=512, kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
trainL=[]

for i in train_label:
  if(i == 1):
    trainL.append(np.array([0,0,1]))
  if(i == 2):
    trainL.append(np.array([0,1,0]))
  if(i == 3):
    trainL.append(np.array([1,0,0]))
trainer = np.array([i for i in trainL])


In [ ]:
X_train,X_val, Y_train, Y_val = train_test_split(train_data, trainer, test_size=0.25)

## I did some data augmentation to keep from overfitting on the limited data. I also normalized the features using rescale This should allow the model to converge faster and more efficiently. 

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
train_generator = train_datagen.flow(X_train, Y_train)

In [ ]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation = val_datagen.flow(X_val,Y_val)

In [ ]:
model.fit(train_generator, validation_data=validation,epochs=25,batch_size=60)
tf.keras.backend.clear_session()



Epoch 1/25
124/124 [==============================] - 23s 154ms/step - loss: 1.0145 - accuracy: 0.4785 - val_loss: 0.7739 - val_accuracy: 0.6730
Epoch 2/25
124/124 [==============================] - 18s 143ms/step - loss: 0.7375 - accuracy: 0.6634 - val_loss: 0.6602 - val_accuracy: 0.7176
Epoch 3/25
124/124 [==============================] - 18s 141ms/step - loss: 0.6533 - accuracy: 0.7184 - val_loss: 0.5111 - val_accuracy: 0.7896
Epoch 4/25
124/124 [==============================] - 18s 143ms/step - loss: 0.5588 - accuracy: 0.7608 - val_loss: 0.5151 - val_accuracy: 0.7790
Epoch 5/25
124/124 [==============================] - 18s 142ms/step - loss: 0.5349 - accuracy: 0.7471 - val_loss: 0.5023 - val_accuracy: 0.7691
Epoch 6/25
124/124 [==============================] - 17s 139ms/step - loss: 0.5220 - accuracy: 0.7706 - val_loss: 0.4698 - val_accuracy: 0.8002
Epoch 7/25
124/124 [==============================] - 18s 141ms/step - loss: 0.5182 - accuracy: 0.7680 - val_loss: 0.4677 - val_ac

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
predi = model.predict(X_val)

In [ ]:
predict = []
for i in predi:
  if(np.argmax(i) == 0):
    predict.append(3)
  if(np.argmax(i) == 1):
    predict.append(2)
  if(np.argmax(i) == 2):
    predict.append(1)


In [ ]:
Y_val

true = []
for i in Y_val:
  if(np.argmax(i) == 0):
    true.append(3)
  if(np.argmax(i) == 1):
    true.append(2)
  if(np.argmax(i) == 2):
    true.append(1)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(true, predict)

0.7547312641937926

In [ ]:
predi = model.predict(test_data)

In [ ]:
predict = []
for i in predi:
  if(np.argmax(i) == 0):
    predict.append(3)
  if(np.argmax(i) == 1):
    predict.append(2)
  if(np.argmax(i) == 2):
    predict.append(1)


In [ ]:
prediction = pd.DataFrame(predict)

In [ ]:
prediction = pd.concat([pd.DataFrame(keys), prediction], axis=1)
prediction.columns = ['id', 'type']

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
prediction

,id,type
0,5309,2
1,5310,1
2,5311,1
3,5312,1
4,5313,1
5,5314,1
6,5315,3
7,5316,1
8,5317,1
9,5318,1


In [ ]:
prediction.to_csv("./prediction.csv", index=False)

Kaggle USername: josedelagarza01